In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc
import glob

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [8]:
data_type = "train"
# month = "07"
# category = "잔액정보"

# local
root_path = '../data/open'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

drive_folder = f'{root_path}/{data_type}/5.잔액정보/'

In [11]:
# 잔액정보 (모든 월 통합) 통합 parquet
balance = pd.read_parquet(f'{drive_folder}train_잔액정보_통합.parquet')
balance

,기준년월,ID,잔액_일시불_B0M,잔액_할부_B0M,잔액_현금서비스_B0M,잔액_리볼빙일시불이월_B0M,잔액_리볼빙CA이월_B0M,잔액_카드론_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,...,평잔_6M,평잔_일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_6M,평잔_RV일시불_해외_6M,평잔_할부_6M,평잔_할부_해외_6M,평잔_CA_6M,평잔_CA_해외_6M,평잔_카드론_6M
0,201807,TRAIN_000000,998,962,22971,0,0,0,1084,547,...,15988,2440,0,0,0,572,0,17008,0,0
1,201807,TRAIN_000001,2565,2390,0,0,0,0,4090,2553,...,7045,2677,0,2830,0,2736,0,0,0,0
2,201807,TRAIN_000002,5312,5113,21531,6795,0,0,5006,8778,...,66549,9118,0,8870,0,4429,0,43351,0,0
3,201807,TRAIN_000003,730,5025,26284,0,0,0,487,5607,...,30139,884,0,0,0,5097,0,30697,0,0
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,28,21,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399996,201812,TRAIN_399996,3351,0,0,0,0,27337,4412,0,...,67826,12524,0,0,0,0,0,0,0,23031
2399997,201812,TRAIN_399997,2524,2960,0,0,0,0,2694,3374,...,8627,3241,0,0,0,3995,0,0,0,0
2399998,201812,TRAIN_399998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# 메모리 사용량이 너무 많아서, int64를 int32로 바꾼다.
for col in balance.select_dtypes(include='int64').columns:
    balance[col] = balance[col].astype('int32')

In [4]:
# 출력 옵션 조정 (생략 없이 dtypes 포함 모든 열 보기)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

In [5]:
# 🔎 결측치 확인
# print("🔍 [결측치 개수]")
# print("▶ balance:\n", balance.isnull().sum().sort_values(ascending=False))

- 연체일자_BOM만 결측치가 있는 것으로 확인했음.

In [7]:
# 연체일자_BOM는 거의 결측치
# 묻지도 않고 삭제하고 살펴보겠음.
balance.drop(columns=['연체일자_B0M'], inplace=True)

In [8]:
# 🔎 데이터 타입
# print("▶ balance dtypes\n", balance.dtypes)

- ID만 object고 , 나머지는 다 int64 --> int32

In [10]:
# 🔎 빈 문자열 확인
# print("\n🔍 [빈 문자열 개수]")
# print("▶ balance 빈 문자열:\n", (balance == '').sum())

- 빈 문자열은 없음

In [12]:
balance.describe().T[['min', 'max']].sort_values('min')

,min,max
연체일수_최근,-999999.0,1.0
연체일수_B1M,-999999.0,1.0
연체일수_B2M,-999999.0,1.0
최종연체회차,-99.0,0.0
RV_평균잔액_R6M,0.0,84969.0
월중평잔_RV일시불,0.0,183116.0
월중평잔_일시불,0.0,223695.0
월중평잔,0.0,330702.0
연체잔액_CA_해외_B0M,0.0,0.0
연체잔액_CA_B0M,0.0,10848.0


- 이상치
- 연체일수_최근 , 연체일수_B1M , 연체일수_B2M 의 최솟값이 -999999 
- 위 3개는 unique() 했을 때, -999999 , 0 , 1 의 값을 가진다.

- 최종연체회차의 최솟값이 -99
- 최종연체회차는 -99와 0의 값을 가진다.

In [14]:
cols = ['연체일수_최근', '연체일수_B1M', '연체일수_B2M', '최종연체회차']

for col in cols:
    print(balance[col].value_counts().sort_index())

연체일수_최근
-999999    2362141
 0           35026
 1            2833
Name: count, dtype: int64
연체일수_B1M
-999999    2121871
 0          277067
 1            1062
Name: count, dtype: int64
연체일수_B2M
-999999    2111112
 0          287883
 1            1005
Name: count, dtype: int64
최종연체회차
-99    1598239
 0      801761
Name: count, dtype: int64


In [15]:
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

In [16]:
# -999999은 데이터가 없는 것을 뜻하니, 0으로 대체
cols = ['연체일수_최근', '연체일수_B1M', '연체일수_B2M']
balance[cols] = balance[cols].replace(-999999, 0)

In [17]:
# 최종연체회차는 -99, 0 만 존재한다.
# -99는 연체정보가 없으므로 0으로 바꿔주고
# 0은 연체 정보가 있는 고객으로 간주하여 1로 바꿔준다.
# 연체정보_유무로 파생변수를 만든다.

balance['연체정보_유무'] = (balance['최종연체회차'] != -99).astype(int)

In [18]:
cols = ['연체일수_최근', '연체일수_B1M', '연체일수_B2M', '최종연체회차', '연체정보_유무']

for col in cols:
    print(balance[col].value_counts().sort_index())

연체일수_최근
0    2397167
1       2833
Name: count, dtype: int64
연체일수_B1M
0    2398938
1       1062
Name: count, dtype: int64
연체일수_B2M
0    2398995
1       1005
Name: count, dtype: int64
최종연체회차
-99    1598239
 0      801761
Name: count, dtype: int64
연체정보_유무
0    1598239
1     801761
Name: count, dtype: int64


- 연체일수_최근, 연체일수_B1M, 연체일수_B2M은 일수 보다는 여부로 보는게 나을지도..

In [20]:
# 유일값 개수가 1개 이하인 컬럼 찾기 (전부 0이거나 전부 NaN)
constant_cols = [col for col in balance.columns if balance[col].nunique(dropna=False) <= 1]

print("🧊 모든 값이 동일한 컬럼 (전부 0이거나 한 값만 있음):")
print(constant_cols)

🧊 모든 값이 동일한 컬럼 (전부 0이거나 한 값만 있음):
['카드론잔액_최종경과월', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M', 'RV잔액이월횟수_R3M', '연체잔액_일시불_해외_B0M', '연체잔액_RV일시불_해외_B0M', '연체잔액_할부_해외_B0M', '연체잔액_CA_해외_B0M']


In [21]:
balance['연체잔액_CA_해외_B0M'].unique()

array([0])

In [22]:
# constant_cols에 있는 컬럼들은 모두 0의 값으로만 이루어져 있으므로,
# 컬럼을 삭제한다.
balance.drop(columns=constant_cols, inplace=True)

In [24]:
# Segment 컬럼 가져와서 balance에 컬럼 붙이기
# 회원정보 7~12월 Segment가 모두 동일한거 확인했음.

# 회원정보에 있는 Segment 불러오기
member = pd.read_parquet('..data/open/train/1.회원정보/201807_train_회원정보.parquet')
member = member[['ID', 'Segment']]  # 필요한 컬럼만

# 잔액정보와 병합
balance = balance.merge(member, on='ID', how='left')

In [25]:
balance['Segment'].value_counts()

Segment
E    1922052
D     349242
C     127590
A        972
B        144
Name: count, dtype: int64

In [27]:
balance

,기준년월,ID,잔액_일시불_B0M,잔액_할부_B0M,잔액_현금서비스_B0M,잔액_리볼빙일시불이월_B0M,잔액_리볼빙CA이월_B0M,잔액_카드론_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,...,평잔_일시불_해외_6M,평잔_RV일시불_6M,평잔_RV일시불_해외_6M,평잔_할부_6M,평잔_할부_해외_6M,평잔_CA_6M,평잔_CA_해외_6M,평잔_카드론_6M,연체정보_유무,Segment
0,201807,TRAIN_000000,998,962,22971,0,0,0,1084,547,...,0,0,0,572,0,17008,0,0,1,D
1,201807,TRAIN_000001,2565,2390,0,0,0,0,4090,2553,...,0,2830,0,2736,0,0,0,0,1,E
2,201807,TRAIN_000002,5312,5113,21531,6795,0,0,5006,8778,...,0,8870,0,4429,0,43351,0,0,1,C
3,201807,TRAIN_000003,730,5025,26284,0,0,0,487,5607,...,0,0,0,5097,0,30697,0,0,0,D
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
2399996,201812,TRAIN_399996,3351,0,0,0,0,27337,4412,0,...,0,0,0,0,0,0,0,23031,0,D
2399997,201812,TRAIN_399997,2524,2960,0,0,0,0,2694,3374,...,0,0,0,3995,0,0,0,0,0,C
2399998,201812,TRAIN_399998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E


In [29]:
# csv너무 느려서 parquet으로 다시 . .
balance.to_parquet(f'{drive_folder}train_잔액정보_통합_전처리1.parquet', index=False)

In [33]:
# 잔액정보 (모든 월 통합) 통합 parquet
balance_eda = pd.read_parquet(f'{drive_folder}train_잔액정보_통합_전처리1.parquet')
balance_eda

,기준년월,ID,잔액_일시불_B0M,잔액_할부_B0M,잔액_현금서비스_B0M,잔액_리볼빙일시불이월_B0M,잔액_리볼빙CA이월_B0M,잔액_카드론_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,...,평잔_일시불_해외_6M,평잔_RV일시불_6M,평잔_RV일시불_해외_6M,평잔_할부_6M,평잔_할부_해외_6M,평잔_CA_6M,평잔_CA_해외_6M,평잔_카드론_6M,연체정보_유무,Segment
0,201807,TRAIN_000000,998,962,22971,0,0,0,1084,547,...,0,0,0,572,0,17008,0,0,1,D
1,201807,TRAIN_000001,2565,2390,0,0,0,0,4090,2553,...,0,2830,0,2736,0,0,0,0,1,E
2,201807,TRAIN_000002,5312,5113,21531,6795,0,0,5006,8778,...,0,8870,0,4429,0,43351,0,0,1,C
3,201807,TRAIN_000003,730,5025,26284,0,0,0,487,5607,...,0,0,0,5097,0,30697,0,0,0,D
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
2399996,201812,TRAIN_399996,3351,0,0,0,0,27337,4412,0,...,0,0,0,0,0,0,0,23031,0,D
2399997,201812,TRAIN_399997,2524,2960,0,0,0,0,2694,3374,...,0,0,0,3995,0,0,0,0,0,C
2399998,201812,TRAIN_399998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
